# Looking for PlanetImagery

Based on https://developers.planet.com/docs/quickstart/searching-for-imagery/

Other: 
- https://developers.planet.com/docs/quickstart/best-practices-large-aois/
-  https://developers.planet.com/docs/quickstart/downloading-imagery/

In [1]:
import requests
from mapboxgl.viz import *

from requests.auth import HTTPBasicAuth

In [2]:
# Get API KEY over
# https://www.planet.com/account/#/
PL_API_KEY = "41321948d404418e94a751df82f7859c"

In [3]:
aoi_url = "https://8ib71h0627.execute-api.us-east-1.amazonaws.com/v1/sites"
geojson = requests.get(aoi_url).json()

features = [
    {
        'type': 'Feature',
        'geometry': site["polygon"],
        'bbox': site["bounding_box"],
        'properties': {"id": site["id"], "label": site["label"]}
    }
    for site in geojson["sites"]
]

geojson = {
    "type": "FeatureCollection",
    "features": features
}

In [4]:
viz = LinestringViz(
    geojson,
    line_width_default=4,
    zoom=5,
)
viz.show()

/Users/vincentsarago/Workspace/venv/py37/lib/python3.7/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### List available product and make sure we can access PlanetScope and SkySat

In [5]:
auth = requests.get("https://api.planet.com/data/v1/item-types", headers={"Authorization": f"api-key {PL_API_KEY}"}).json()
my_items = [item["id"] for item in auth["item_types"]]
# We need to discover Skysat and Planetscope coverage:
assert "PSOrthoTile" in my_items
assert "SkySatScene" in my_items

print(my_items)

['PSOrthoTile', 'REOrthoTile', 'PSScene3Band', 'PSScene4Band', 'REScene', 'Landsat8L1G', 'Sentinel2L1C', 'SkySatScene', 'SkySatCollect', 'SkySatVideo', 'Sentinel1', 'MOD09GA', 'MOD09GQ', 'MYD09GA', 'MYD09GQ']


### Helper functions (Stats and Search)

In [6]:
def stats(geometry, collections=["SkySatScene", "PSOrthoTile"]):
    """Retrieve Stats"""
    # filter for items the overlap with our chosen geometry
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geometry
    }

    # filter images acquired in a certain date range
    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": "2020-01-01T00:00:00.000Z",
      }
    }

    # filter any images which are more than 50% clouds
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.05
      }
    }

    config = {
      "type": "AndFilter",
      "config": [geometry_filter, cloud_cover_filter, date_range_filter]
    }
    
    # Stats API request object
    stats_endpoint_request = {
      "interval": "day", "item_types": collections, "filter": config
    }

    # fire off the POST request
    result = requests.post('https://api.planet.com/data/v1/stats', auth=HTTPBasicAuth(PL_API_KEY, ''), json=stats_endpoint_request)
    return result.json()


def search(geometry, collections=["SkySatScene", "PSOrthoTile"]):
    """Search for Data."""
    # filter for items the overlap with our chosen geometry
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geometry
    }

    # filter images acquired in a certain date range
    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": "2020-01-01T00:00:00.000Z",
      }
    }

    # filter any images which are more than 50% clouds
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.05
      }
    }

    config = {
      "type": "AndFilter",
      "config": [geometry_filter, cloud_cover_filter, date_range_filter]
    }
    
    # Stats API request object
    stats_endpoint_request = {
      "interval": "day",
      "item_types": collections,
      "filter": config
    }

    # fire off the POST request
    result = requests.post('https://api.planet.com/data/v1/quick-search', auth=HTTPBasicAuth(PL_API_KEY, ''), json=stats_endpoint_request)
    return result.json()


## Stats for SkySatScene

In [7]:
results = {feat["properties"]["label"] : stats(feat["geometry"], collections=["SkySatScene"]) for feat in geojson["features"]}

print("Total of SkySatScene scenes per sites")
for k, r in results.items():
    print(k)
    im = r.get("buckets", [])
    total = sum([f["count"] for f in im])
    print(total)
    print("---")

Total of SkySatScene scenes per sites
Beijing
4450
---
Port of Dunkirk
72
---
Port of Ghent
24
---
Los Angeles
6247
---
San Francisco
9117
---
Tokyo
5229
---


## Stats for PSOrthoTile

In [8]:
results = {feat["properties"]["label"] : stats(feat["geometry"], collections=["PSOrthoTile"]) for feat in geojson["features"]}

print("Total of PSOrthoTile scenes per sites")
for k, r in results.items():
    print(k)
    im = r.get("buckets", [])
    total = sum([f["count"] for f in im])
    print(total)
    print("---")

Total of PSOrthoTile scenes per sites
Beijing
3826
---
Port of Dunkirk
164
---
Port of Ghent
168
---
Los Angeles
3816
---
San Francisco
3813
---
Tokyo
1532
---


In [9]:
results_skySat = {feat["properties"]["label"] : search(feat["geometry"], collections=["SkySatScene"]) for feat in geojson["features"]}
results_PS = {feat["properties"]["label"] : search(feat["geometry"], collections=["PSOrthoTile"]) for feat in geojson["features"]}

## Beijing

In [10]:
bounds = geojson["features"][0]["bbox"]
center = ((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2)

# SkySat
images = {
    "type": "FeatureCollection",
    "features": results_skySat["Beijing"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

# PlanetScope
images = {
    "type": "FeatureCollection",
    "features": results_PS["Beijing"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

## Port of Dunkirk

In [11]:
bounds = geojson["features"][1]["bbox"]
center = ((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2)

# SkySat
images = {
    "type": "FeatureCollection",
    "features": results_skySat["Port of Dunkirk"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

# PlanetScope
images = {
    "type": "FeatureCollection",
    "features": results_PS["Port of Dunkirk"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

## Port of Ghent

In [12]:
bounds = geojson["features"][2]["bbox"]
center = ((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2)

# SkySat
images = {
    "type": "FeatureCollection",
    "features": results_skySat["Port of Ghent"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

# PlanetScope
images = {
    "type": "FeatureCollection",
    "features": results_PS["Port of Ghent"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

## Los Angeles

In [13]:
bounds = geojson["features"][3]["bbox"]
center = ((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2)

# SkySat
images = {
    "type": "FeatureCollection",
    "features": results_skySat["Los Angeles"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

# PlanetScope
images = {
    "type": "FeatureCollection",
    "features": results_PS["Los Angeles"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

### San Francisco

In [14]:
bounds = geojson["features"][4]["bbox"]
center = ((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2)

# SkySat
images = {
    "type": "FeatureCollection",
    "features": results_skySat["San Francisco"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

# PlanetScope
images = {
    "type": "FeatureCollection",
    "features": results_PS["San Francisco"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

## Tokyo

In [15]:
bounds = geojson["features"][5]["bbox"]
center = ((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2)

# SkySat
images = {
    "type": "FeatureCollection",
    "features": results_skySat["Tokyo"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

# PlanetScope
images = {
    "type": "FeatureCollection",
    "features": results_PS["Tokyo"]["features"]
}
viz = LinestringViz(images, line_width_default=4, zoom=7, center=center)
viz.show()

### NEXT: https://developers.planet.com/docs/quickstart/downloading-imagery/

In [124]:
# Get images Id 
feats = results_skySat["San Francisco"]["features"]

In [126]:
[f["id"] for f in feats]

['20200523_214815_ssc6d2_0013',
 '20200523_214815_ssc6d2_0012',
 '20200523_214815_ssc6d2_0011',
 '20200523_214815_ssc6d2_0010',
 '20200523_214815_ssc6d1_0012',
 '20200523_214815_ssc6d1_0008',
 '20200523_214815_ssc6d1_0007',
 '20200523_214815_ssc6d1_0006',
 '20200523_214815_ssc6d1_0005',
 '20200523_214815_ssc6d1_0004',
 '20200523_214815_ssc6d1_0003',
 '20200523_213223_ssc11d1_0028',
 '20200523_213223_ssc11d1_0027',
 '20200523_213223_ssc11d1_0026',
 '20200523_213223_ssc11d1_0025',
 '20200523_184557_ssc4d3_0012',
 '20200523_184557_ssc4d3_0011',
 '20200523_184557_ssc4d3_0010',
 '20200523_184557_ssc4d3_0009',
 '20200523_184557_ssc4d3_0008',
 '20200523_184557_ssc4d3_0007',
 '20200523_184557_ssc4d3_0006',
 '20200523_184557_ssc4d3_0005',
 '20200523_184557_ssc4d3_0004',
 '20200523_184557_ssc4d2_0005',
 '20200523_184557_ssc4d2_0004',
 '20200523_184557_ssc4d2_0003',
 '20200523_184557_ssc4d2_0002',
 '20200522_214939_ssc10d2_0096',
 '20200522_214939_ssc10d2_0095',
 '20200522_214939_ssc10d2_0094',
 